In [1]:
import seaborn as sns
import json
import befaas.logentry as le
from befaas.fileutil import log_entry_iterator

sns.set()

filepath = "../dumps/"
dump = "dumpIoT4.json"
outFile = "../dumps/uniqueIoT.json"
print("Everything is set up.")


Everything is set up.


In [2]:
print(f"Include dump {dump} ...")
# Count entries to get some progress feedback
entries = 0
contexts = {}
otherUserAgentContextList = []

for entry in log_entry_iterator(filepath + dump):
    entries = entries + 1
    if (entries % 10000 == 0):
        print(f"(I)  processed {entries} entries")

    logentry = le.cast_log_type(entry)
    if isinstance(logentry, le.RequestLog):
        userAgent = logentry.event['request']['headers']['user-agent']
        if not str(userAgent).startswith("node-fetch") | str(userAgent).startswith("Artillery"):
            print(f"Other user agent, will filter: {userAgent}")
            otherUserAgentContextList.append(logentry.context_id)

entries = 0
duplicates = 0
print(f"Writing to {outFile}")
with open(outFile, "w") as jsfile:
    jsfile.write("[")
    initial = True

    for entry in log_entry_iterator(filepath + dump):
        entries = entries + 1
        entryOK = True
        if (entries % 10000 == 0):
            print(f"(II) processed {entries} entries")

        logentry = le.cast_log_type(entry)
        id = logentry.context_id

        if id in otherUserAgentContextList:
            # Wrong user agent
            print(f"filtered context {id}")
            entryOK = False
            continue

        # Create context if there isn't one
        if not (id in contexts):
            contexts[id] = []

        for checkEntry in contexts[id]:
            if str(checkEntry).strip() == str(entry).strip():
                # found duplicate
                duplicates += 1
                entryOK = False
                #print(f"filtered a duplicate for ctx {id}")
                continue

        if (entryOK):
            contexts[id].append(entry)
            #write entry to file
            jsfile.write(("" if initial else ",\n") + json.dumps(entry))
            initial = False

    jsfile.write("]")

print(f"Found and removed {duplicates} duplicates.")

Include dump dumpIoT4.json ...
Iterate over ../dumps/dumpIoT4.json ...
(I)  processed 10000 entries
(I)  processed 20000 entries
(I)  processed 30000 entries
(I)  processed 40000 entries
(I)  processed 50000 entries
(I)  processed 60000 entries
(I)  processed 70000 entries
(I)  processed 80000 entries
(I)  processed 90000 entries
(I)  processed 100000 entries
(I)  processed 110000 entries
(I)  processed 120000 entries
(I)  processed 130000 entries
(I)  processed 140000 entries
(I)  processed 150000 entries
(I)  processed 160000 entries
(I)  processed 170000 entries
(I)  processed 180000 entries
(I)  processed 190000 entries
(I)  processed 200000 entries
(I)  processed 210000 entries
(I)  processed 220000 entries
(I)  processed 230000 entries
(I)  processed 240000 entries
(I)  processed 250000 entries
(I)  processed 260000 entries
(I)  processed 270000 entries
(I)  processed 280000 entries
(I)  processed 290000 entries
(I)  processed 300000 entries
(I)  processed 310000 entries
(I)  pro